In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm.notebook import tqdm # to avoid duplicated prints

#### Inputs:
- Sirene : Fichier StockEtablissementHistorique du 01 mars 2020 [(URL)](https://www.data.gouv.fr/fr/datasets/r/14a5d3e0-b7cf-4aa6-b5fe-c64854727388)
- Documentation des variables [(URL)](https://static.data.gouv.fr/resources/base-sirene-des-entreprises-et-de-leurs-etablissements-siren-siret/20191126-150732/description-fichier-stocketablissement.pdf)

Seules les données courantes sont disponible. Les données historisées de l'INSEE ne proposent pas la tranche d'effectif. De plus, une données d'effectif n'est pas renseignée quand elle est vieille d'au moins trois ans. On s'intéresse aux entreprises ayant au moins un effectif renseigné à une date postérieure à 2017, et on fait l'hypothèse que les entreprises de plus de 500 employés aujourd'hui (2019 et 2020) en avaient déjà plus de 500 en 2018. Et que les entreprises de plus de 500 employés aujourd'hui n'ayant pas renseignée leurs effectifs sont une minorité.

In [7]:
# filters and mapping
def above_date(annee, min_annee=2017):
    try:
        return annee>=min_annee
    except:
        return False
def above_500_employees(tranche):
    if int(tranche)>=40:
        return True
    else:
        return False

def not_an_administration(naf_code):
    # Exclude section O = public administration and P = Teaching of French NAF
    # https://www.insee.fr/fr/metadonnees/cpfr21/section/O?champRecherche=false
    try:
        resp = not (naf_code.startswith("84") or naf_code.startswith("85"))
        return resp
    except:
        return True
    
tranches = {41 : "500 à 999 salariés",
            42 : "1 000 à 1 999 salariés",
            51 : "2 000 à 4 999 salariés",
            52 : "5 000 à 9 999 salariés",
            53 : "10 000 salariés et plus"}

# read insee database by chunk because it is large
chunk_reader = pd.read_csv("../../data/input/Entreprises/INSEE/StockEtablissement_utf8.csv",
                 usecols=["siren",
                          "activitePrincipaleEtablissement",
                          "anneeEffectifsEtablissement",
                          "trancheEffectifsEtablissement"],
                 chunksize=100001,
               na_values="NN",
                dtype= {"siren": str,
                        "activitePrincipaleEtablissement":str,
                         "anneeEffectifsEtablissement": np.float64,
                         "trancheEffectifsEtablissement": np.float64})
big_companies = []
for df in tqdm(chunk_reader): # 297 expected
    df = df[df["anneeEffectifsEtablissement"].apply(above_date)]
    df = df[df["trancheEffectifsEtablissement"].apply(above_500_employees)]
    try:
        df = df[df["activitePrincipaleEtablissement"].apply(not_an_administration)]
    except:
        print("Activity could not be filtered - this happens for last chunk of the data...")
    big_companies.append(df)

# group the data chunks
df = pd.concat(big_companies,axis=0, ignore_index=True)
df = df.groupby("siren").agg({"trancheEffectifsEtablissement":max, "anneeEffectifsEtablissement":max})
df = df.reset_index()
df["trancheEffectifsEtablissement_texte"] = df["trancheEffectifsEtablissement"].replace(tranches)
df = df[["siren",
         "anneeEffectifsEtablissement",
         "trancheEffectifsEtablissement_texte",
         "trancheEffectifsEtablissement"]]
# save (default encoding is utf-8)
df.to_csv("../../data/processed/Entreprises/insee_entreprises_effectifs_sup_500_post_2017.csv", 
          sep=";", 
          index=False)

Activity could not be filtered
Activity could not be filtered
Activity could not be filtered
Activity could not be filtered
Activity could not be filtered
Activity could not be filtered
Activity could not be filtered
Activity could not be filtered
Activity could not be filtered
Activity could not be filtered
Activity could not be filtered

